### Import Required Libraries and Set Up Environment Variables

In [18]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json
import os
from datetime import datetime

## Load the NASA_API_KEY from the env file
load_dotenv()
NASA_API_KEY = os.getenv('NASA_API_KEY')

print(NASA_API_KEY)

sVwG0HP2zbvOvwII7sKPi6PMiRNmIt2gc1cmuNWJ


### CME Data

In [19]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for CMEs:
CME = "CME"

# Search for CMEs published between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for CME
cme_url = f"{base_url}{CME}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

# Display the generated URL for the CME search query
print(cme_url)

https://api.nasa.gov/DONKI/CME?startDate=2013-05-01&endDate=2024-05-01&api_key=sVwG0HP2zbvOvwII7sKPi6PMiRNmIt2gc1cmuNWJ


In [20]:
# Make a "GET" request for the CME URL and store it in a variable named cme_response
cme_response = requests.get(cme_url, timeout=400)

# Check if the request was successful
if cme_response.status_code == 200:
    cme_json = cme_response.json()  # Parse the JSON response
    # print(json.dumps(cme_data[:5], indent=2))  # Pretty-print the response
    print("CME data successfully retrieved and stored.") 
else:
    print(f"Error: Unable to retrieve data, status code {cme_response.status_code}")


CME data successfully retrieved and stored.


In [21]:
# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
if cme_json:
    print(json.dumps(cme_json[0], indent=4))
else:
    print("No data found")

{
    "activityID": "2013-05-01T03:12:00-CME-001",
    "catalog": "M2M_CATALOG",
    "startTime": "2013-05-01T03:12Z",
    "instruments": [
        {
            "displayName": "SOHO: LASCO/C2"
        },
        {
            "displayName": "SOHO: LASCO/C3"
        },
        {
            "displayName": "STEREO A: SECCHI/COR2"
        },
        {
            "displayName": "STEREO B: SECCHI/COR2"
        }
    ],
    "sourceLocation": "",
    "activeRegionNum": null,
    "note": "",
    "submissionTime": "2013-08-07T16:54Z",
    "versionId": 1,
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/CME/2349/-1",
    "cmeAnalyses": [
        {
            "isMostAccurate": true,
            "time21_5": "2013-05-01T07:07Z",
            "latitude": 12.0,
            "longitude": -120.0,
            "halfAngle": 36.0,
            "speed": 860.0,
            "type": "C",
            "featureCode": "null",
            "imageType": null,
            "measurementTechnique": "null",
   

In [22]:
# Convert cme_json to a Pandas DataFrame 
df_cme = pd.DataFrame(cme_json)

# Keep only the columns: activityID, startTime, linkedEvents
df_cme_filtered = df_cme[['activityID', 'startTime', 'linkedEvents']]

# Preview the first few rows of the filtered DataFrame
print(df_cme_filtered.head())

                    activityID          startTime  \
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
1  2013-05-02T05:24:00-CME-001  2013-05-02T05:24Z   
2  2013-05-02T14:36:00-CME-001  2013-05-02T14:36Z   
3  2013-05-03T18:00:00-CME-001  2013-05-03T18:00Z   
4  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   

                                      linkedEvents  
0  [{'activityID': '2013-05-04T04:52:00-IPS-001'}]  
1                                             None  
2                                             None  
3                                             None  
4  [{'activityID': '2013-05-07T04:37:00-IPS-001'}]  


In [23]:
# Notice that the linkedEvents column allows us to identify the corresponding GST
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to GSTs
df_cme_filtered_cleaned = df_cme_filtered.dropna(subset=['linkedEvents'])

#print(df_cme_filtered_cleaned.head())
display(df_cme_filtered_cleaned.head())

,activityID,startTime,linkedEvents
0,2013-05-01T03:12:00-CME-001,2013-05-01T03:12Z,[{'activityID': '2013-05-04T04:52:00-IPS-001'}]
4,2013-05-03T22:36:00-CME-001,2013-05-03T22:36Z,[{'activityID': '2013-05-07T04:37:00-IPS-001'}]
7,2013-05-09T19:29:00-CME-001,2013-05-09T19:29Z,[{'activityID': '2013-05-12T23:30:00-IPS-001'}]
10,2013-05-13T02:54:00-CME-001,2013-05-13T02:54Z,[{'activityID': '2013-05-13T01:53:00-FLR-001'}...
13,2013-05-13T16:18:00-CME-001,2013-05-13T16:18Z,[{'activityID': '2013-05-13T15:40:00-FLR-001'}...


In [24]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Write a nested for loop that iterates first over each row in the cme DataFrame (using the index)
# and then iterates over the values in 'linkedEvents' 
# and adds the elements individually to a list of dictionaries where each row is one element 

# Initialize an empty list to store the expanded rows
expanded_rows = []

# Iterate over each index in the DataFrame
for i in df_cme_filtered_cleaned.index:
    # Get the corresponding value from row i in 'activityID'
    activityID = df_cme_filtered_cleaned.at[i, 'activityID']
    
    # Get the corresponding value from row i in 'startTime'
    startTime = df_cme_filtered_cleaned.at[i, 'startTime']
    
    # Get the list of dictionaries from row i in 'linkedEvents'
    linkedEvents = df_cme_filtered_cleaned.at[i, 'linkedEvents']

    # Iterate over each dictionary in the list
    for item in linkedEvents:
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
        expanded_rows.append({
            'activityID': activityID,
            'startTime': startTime,
            #'eventType': item['eventType'],  # Extract eventType from the dictionary
            'eventID': item['activityID']    # Extract activityID (eventID) from the dictionary
        })

    
        # Append a new dictionary to the expanded_rows list for each dictionary item and corresponding 'activityID' and 'startTime' value
      
# Create a new DataFrame from the expanded rows
df_expanded = pd.DataFrame(expanded_rows)

print(df_expanded.head())

                    activityID          startTime                      eventID
0  2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z  2013-05-04T04:52:00-IPS-001
1  2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z  2013-05-07T04:37:00-IPS-001
2  2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z  2013-05-12T23:30:00-IPS-001
3  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z  2013-05-13T01:53:00-FLR-001
4  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z  2013-05-13T04:12:00-SEP-001


In [25]:
# Create a function called extract_activityID_from_dict that takes a dict as input such as in linkedEvents
# and verify below that it works as expected using one row from linkedEvents as an example
# Be sure to use a try and except block to handle errors

        # Log the error or print it for debugging

def extract_activityID_from_dict(input_dict):
    try:
        # Attempt to extract the 'activityID' from the dictionary
        return input_dict['activityID']
    except (KeyError, TypeError) as e:
        # If 'activityID' is not found or input_dict is not a dictionary, return None
        return None

# Test the function on a specific row in the DataFrame
result = extract_activityID_from_dict(df_cme_filtered_cleaned.loc[0, 'linkedEvents'][0])
print(result)


2013-05-04T04:52:00-IPS-001


In [26]:
# Apply this function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'GST_ActivityID' using loc indexer:
df_cme_filtered_cleaned = df_cme_filtered_cleaned.copy()


df_cme_filtered_cleaned.loc[:, 'GST_ActivityID'] = df_cme_filtered_cleaned['linkedEvents'].apply(
    lambda linked_events: extract_activityID_from_dict(linked_events[0]) if linked_events and isinstance(linked_events, list) else None
)

In [27]:
# Remove rows with missing GST_ActivityID, since we can't assign them to GSTs:
df_cme_filtered_cleaned = df_cme_filtered_cleaned.dropna(subset=['GST_ActivityID'])

print(df_cme_filtered_cleaned.head())

                     activityID          startTime  \
0   2013-05-01T03:12:00-CME-001  2013-05-01T03:12Z   
4   2013-05-03T22:36:00-CME-001  2013-05-03T22:36Z   
7   2013-05-09T19:29:00-CME-001  2013-05-09T19:29Z   
10  2013-05-13T02:54:00-CME-001  2013-05-13T02:54Z   
13  2013-05-13T16:18:00-CME-001  2013-05-13T16:18Z   

                                         linkedEvents  \
0     [{'activityID': '2013-05-04T04:52:00-IPS-001'}]   
4     [{'activityID': '2013-05-07T04:37:00-IPS-001'}]   
7     [{'activityID': '2013-05-12T23:30:00-IPS-001'}]   
10  [{'activityID': '2013-05-13T01:53:00-FLR-001'}...   
13  [{'activityID': '2013-05-13T15:40:00-FLR-001'}...   

                 GST_ActivityID  
0   2013-05-04T04:52:00-IPS-001  
4   2013-05-07T04:37:00-IPS-001  
7   2013-05-12T23:30:00-IPS-001  
10  2013-05-13T01:53:00-FLR-001  
13  2013-05-13T15:40:00-FLR-001  


In [28]:
# print out the datatype of each column in this DataFrame:
display(df_cme_filtered_cleaned.dtypes)


activityID        object
startTime         object
linkedEvents      object
GST_ActivityID    object
dtype: object

In [29]:
# Convert the 'GST_ActivityID' column to string format 
df_cme_filtered_cleaned['GST_ActivityID'] = df_cme_filtered_cleaned['GST_ActivityID'].astype(str)

df_cme_filtered_cleaned.dtypes
# Convert startTime to datetime format  
df_cme_filtered_cleaned['startTime'] = pd.to_datetime(df_cme_filtered_cleaned['startTime'], errors='coerce')

# Rename startTime to startTime_CME and activityID to cmeID
df_cme_filtered_cleaned.rename(columns={
    'startTime': 'startTime_CME',
    'activityID': 'cmeID'
}, inplace=True)

print(df_cme_filtered_cleaned.columns)
# Drop linkedEvents
df_cme_filtered_cleaned = df_cme_filtered_cleaned.drop(columns=['linkedEvents'])

print(df_cme_filtered_cleaned.columns)
# Verify that all steps were executed correctly


Index(['cmeID', 'startTime_CME', 'linkedEvents', 'GST_ActivityID'], dtype='object')
Index(['cmeID', 'startTime_CME', 'GST_ActivityID'], dtype='object')


In [30]:
# We are only interested in CMEs related to GSTs so keep only rows where the GST_ActivityID column contains 'GST'
# use the method 'contains()' from the str library.  

df_cme_filtered_cleaned = df_cme_filtered_cleaned[df_cme_filtered_cleaned['GST_ActivityID'].str.contains('GST', na=False)]

print(df_cme_filtered_cleaned.head())


                            cmeID             startTime_CME  \
26    2013-06-02T20:24:00-CME-001 2013-06-02 20:24:00+00:00   
1358  2015-09-04T14:12:00-CME-001 2015-09-04 14:12:00+00:00   
1359  2015-09-04T19:24:00-CME-001 2015-09-04 19:24:00+00:00   
1361  2015-09-06T23:30:00-CME-001 2015-09-06 23:30:00+00:00   
1362  2015-09-07T10:36:00-CME-001 2015-09-07 10:36:00+00:00   

                   GST_ActivityID  
26    2013-06-07T03:00:00-GST-001  
1358  2015-09-09T03:00:00-GST-001  
1359  2015-09-09T03:00:00-GST-001  
1361  2015-09-11T06:00:00-GST-001  
1362  2015-09-11T06:00:00-GST-001  


### GST Data

In [32]:
# Set the base URL to NASA's DONKI API:
base_url = "https://api.nasa.gov/DONKI/"

# Set the specifier for Geomagnetic Storms (GST):
GST = "GST"

# Search for GSTs between a begin and end date
startDate = "2013-05-01"
endDate   = "2024-05-01"

# Build URL for GST

gst_url = f"{base_url}{GST}?startDate={startDate}&endDate={endDate}&api_key={NASA_API_KEY}"

# Display the generated URL for the CME search query
print(gst_url)

https://api.nasa.gov/DONKI/GST?startDate=2013-05-01&endDate=2024-05-01&api_key=sVwG0HP2zbvOvwII7sKPi6PMiRNmIt2gc1cmuNWJ


In [33]:
# Make a "GET" request for the GST URL and store it in a variable named gst_response

gst_response = requests.get(gst_url, timeout=400)

# Check if the request was successful
if gst_response.status_code == 200:
    gst_json = gst_response.json()  # Parse the JSON response
    # print(json.dumps(cme_data[:5], indent=2))  # Pretty-print the response
    print("GST data successfully retrieved and stored.") 
else:
    print(f"Error: Unable to retrieve data, status code {gst_response.status_code}")

GST data successfully retrieved and stored.


In [34]:
# Convert the response variable to json and store it as a variable named gst_json

# Preview the first result in JSON format
# Use json.dumps with argument indent=4 to format data
if gst_json:
    print(json.dumps(gst_json[0], indent=4))
else:
    print("No data found")

{
    "gstID": "2013-06-01T01:00:00-GST-001",
    "startTime": "2013-06-01T01:00Z",
    "allKpIndex": [
        {
            "observedTime": "2013-06-01T01:00Z",
            "kpIndex": 6.0,
            "source": "NOAA"
        }
    ],
    "link": "https://webtools.ccmc.gsfc.nasa.gov/DONKI/view/GST/326/-1",
    "linkedEvents": [
        {
            "activityID": "2013-05-31T15:45:00-HSS-001"
        }
    ],
    "submissionTime": "2013-07-15T19:26Z",
    "versionId": 1
}


In [37]:
# Convert gst_json to a Pandas DataFrame  
df_gst = pd.DataFrame(gst_json)

# Keep only the columns: activityID, startTime, linkedEvents
df_gst_filtered = df_gst[['gstID', 'startTime', 'linkedEvents']]

# Preview the first few rows of the filtered DataFrame
df_gst_filtered.head()



,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
2,2013-06-29T03:00:00-GST-001,2013-06-29T03:00Z,None
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...


In [39]:
# Notice that the linkedEvents column allows us to identify the corresponding CME
# Remove rows with missing 'linkedEvents' since we won't be able to assign these to CME
df_gst_filtered_cleaned = df_gst_filtered.dropna(subset=['linkedEvents'])

df_gst_filtered_cleaned


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,[{'activityID': '2013-05-31T15:45:00-HSS-001'}]
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,[{'activityID': '2013-06-02T20:24:00-CME-001'}]
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,[{'activityID': '2013-09-29T22:40:00-CME-001'}...
4,2013-12-08T00:00:00-GST-001,2013-12-08T00:00Z,[{'activityID': '2013-12-04T23:12:00-CME-001'}...
5,2014-02-19T03:00:00-GST-001,2014-02-19T03:00Z,[{'activityID': '2014-02-16T14:15:00-CME-001'}...
...,...,...,...
112,2023-12-18T06:00:00-GST-001,2023-12-18T06:00Z,[{'activityID': '2023-12-14T17:38:00-CME-001'}...
113,2024-03-03T18:00:00-GST-001,2024-03-03T18:00Z,[{'activityID': '2024-02-28T17:48:00-CME-001'}...
114,2024-03-23T21:00:00-GST-001,2024-03-23T21:00Z,[{'activityID': '2024-03-23T12:49:00-IPS-001'}...
115,2024-03-24T12:00:00-GST-001,2024-03-24T12:00Z,[{'activityID': '2024-03-23T01:25:00-CME-001'}...


In [40]:
# Notice that the linkedEvents sometimes contains multiple events per row
# Use the explode method to ensure that each row is one element. Ensure to reset the index and drop missing values.
df_gst_exploded = df_gst_filtered_cleaned.explode('linkedEvents')

df_gst_exploded = df_gst_exploded.reset_index(drop=True).dropna(subset=['linkedEvents'])

df_gst_exploded.head()


,gstID,startTime,linkedEvents
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'}
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'}
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'}
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'}
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'}


In [41]:
# Apply the extract_activityID_from_dict function to each row in the 'linkedEvents' column (you can use apply() and a lambda function)
# and create a new column called 'CME_ActivityID' using loc indexer:

# Remove rows with missing CME_ActivityID, since we can't assign them to CMEs:
def extract_activityID_from_dict(linked_event):
    try:
        # Check if 'activityID' exists in the dictionary and return it
        if 'activityID' in linked_event:
            return linked_event['activityID']
        else:
            return None
    except (ValueError, TypeError):
        return None

df_gst_exploded['CME_ActivityID'] = df_gst_exploded['linkedEvents'].apply(lambda x: extract_activityID_from_dict(x))

df_gst_cleaned = df_gst_exploded.dropna(subset=['CME_ActivityID'])

df_gst_cleaned.head()


,gstID,startTime,linkedEvents,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01T01:00Z,{'activityID': '2013-05-31T15:45:00-HSS-001'},2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07T03:00Z,{'activityID': '2013-06-02T20:24:00-CME-001'},2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-09-29T22:40:00-CME-001'},2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T01:54:00-IPS-001'},2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02T03:00Z,{'activityID': '2013-10-02T02:47:00-MPC-001'},2013-10-02T02:47:00-MPC-001


In [42]:
# Convert the 'CME_ActivityID' column to string format 
df_gst_cleaned['CME_ActivityID'] = df_gst_cleaned['CME_ActivityID'].astype(str)

# Convert the 'gstID' column to string format 
df_gst_cleaned['gstID'] = df_gst_cleaned['gstID'].astype(str)

# Convert startTime to datetime format  
df_gst_cleaned['startTime'] = pd.to_datetime(df_gst_cleaned['startTime'], errors='coerce')

# Rename startTime to startTime_GST 
df_gst_cleaned.rename(columns={'startTime': 'startTime_GST'}, inplace=True)

# Drop linkedEvents
df_gst_cleaned.drop(columns=['linkedEvents'], inplace=True)

# Verify that all steps were executed correctly
df_gst_cleaned.info()  # To check the data types
df_gst_cleaned.head()  # To preview the cleaned DataFrame

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 205 entries, 0 to 204
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype              
---  ------          --------------  -----              
 0   gstID           205 non-null    object             
 1   startTime_GST   205 non-null    datetime64[ns, UTC]
 2   CME_ActivityID  205 non-null    object             
dtypes: datetime64[ns, UTC](1), object(2)
memory usage: 4.9+ KB


,gstID,startTime_GST,CME_ActivityID
0,2013-06-01T01:00:00-GST-001,2013-06-01 01:00:00+00:00,2013-05-31T15:45:00-HSS-001
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
3,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T01:54:00-IPS-001
4,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-10-02T02:47:00-MPC-001


In [43]:
# We are only interested in GSTs related to CMEs so keep only rows where the CME_ActivityID column contains 'CME'
# use the method 'contains()' from the str library.  
df_gst_related_to_cme = df_gst_cleaned[df_gst_cleaned['CME_ActivityID'].str.contains('CME', na=False)]

df_gst_related_to_cme.head()

,gstID,startTime_GST,CME_ActivityID
1,2013-06-07T03:00:00-GST-001,2013-06-07 03:00:00+00:00,2013-06-02T20:24:00-CME-001
2,2013-10-02T03:00:00-GST-001,2013-10-02 03:00:00+00:00,2013-09-29T22:40:00-CME-001
5,2013-12-08T00:00:00-GST-001,2013-12-08 00:00:00+00:00,2013-12-04T23:12:00-CME-001
7,2014-02-19T03:00:00-GST-001,2014-02-19 03:00:00+00:00,2014-02-16T14:15:00-CME-001
9,2014-02-20T03:00:00-GST-001,2014-02-20 03:00:00+00:00,2014-02-18T01:25:00-CME-001


### Merge both datatsets

In [45]:
# Now merge both datasets using 'gstID' and 'CME_ActivityID' for gst and 'GST_ActivityID' and 'cmeID' for cme. Use the 'left_on' and 'right_on' specifiers.

df_merged = pd.merge(
    df_gst_related_to_cme,   # GST DataFrame
    df_cme_filtered_cleaned, # CME DataFrame
    left_on=['gstID', 'CME_ActivityID'],  # Columns from the GST DataFrame
    right_on=['GST_ActivityID', 'cmeID'], # Columns from the CME DataFrame
    how='left'              # Use left join to keep all GST records
)



In [49]:
# Verify that the new DataFrame has the same number of rows as cme and gst

rows_gst = df_gst_related_to_cme.shape[0]
print(f"Number of rows in GST DataFrame: {rows_gst}")

rows_cme = df_cme_filtered_cleaned.shape[0]
print(f"Number of rows in CME DataFrame: {rows_cme}")

rows_merged = df_merged.shape[0]
print(f"Number of rows in the merged DataFrame: {rows_merged}")

if rows_merged == rows_gst or rows_merged == rows_cme:
    print("The merged DataFrame has the correct number of rows.")
else:
    print("The merged DataFrame does not have the same number of rows as GST or CME.")

Number of rows in GST DataFrame: 61
Number of rows in CME DataFrame: 6
Number of rows in the merged DataFrame: 61
The merged DataFrame has the correct number of rows.


### Computing the time it takes for a CME to cause a GST

In [50]:
# Compute the time diff between startTime_GST and startTime_CME by creating a new column called `timeDiff`.

df_merged['timeDiff'] = df_merged['startTime_GST'] - df_merged['startTime_CME']

df_merged[['startTime_GST', 'startTime_CME', 'timeDiff']].head()


,startTime_GST,startTime_CME,timeDiff
0,2013-06-07 03:00:00+00:00,2013-06-02 20:24:00+00:00,4 days 06:36:00
1,2013-10-02 03:00:00+00:00,NaT,NaT
2,2013-12-08 00:00:00+00:00,NaT,NaT
3,2014-02-19 03:00:00+00:00,NaT,NaT
4,2014-02-20 03:00:00+00:00,NaT,NaT


In [53]:
# Use describe() to compute the mean and median time 
# that it takes for a CME to cause a GST. 

time_diff_stats = df_merged['timeDiff'].describe()

mean_time_diff = time_diff_stats['mean']
median_time_diff = df_merged['timeDiff'].median()

print(f"Mean time difference: {mean_time_diff}")
print(f"Median time difference: {median_time_diff}")

Mean time difference: 3 days 18:31:00
Median time difference: 4 days 06:33:00


### Exporting data in csv format

In [54]:
# Export data to CSV without the index
df_merged.to_csv('merged_data.csv', index=False)


In [ ]:
# completed